In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/Mushroom_dataset/mushrooms.csv')

In [3]:
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [4]:
df.shape

(8124, 23)

In [5]:
le = LabelEncoder()

In [6]:
le.fit_transform?

In [7]:
df = df.apply(le.fit_transform)

In [8]:
print(df.shape)

(8124, 23)


In [9]:
data = df.values

In [10]:
print(type(data))

<class 'numpy.ndarray'>


In [11]:
data_X = data[:, 1:]
data_Y = data[:, 0]

In [12]:
print(data_X.shape, data_Y.shape)

(8124, 22) (8124,)


In [13]:
split = 0.8

train_X = data_X[:int(split*data_X.shape[0]), :]
train_Y = data_Y[:int(split*data_Y.shape[0])]
test_X = data_X[int(split*data_X.shape[0]):, :]
test_Y = data_Y[int(split*data_Y.shape[0]):]

In [14]:
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(6499, 22) (6499,) (1625, 22) (1625,)


In [15]:
def prior_prob(Y_train, label):
    return np.sum(Y_train == label)/Y_train.shape[0]

In [18]:
print(prior_prob(train_Y, 1))

0.43052777350361593


In [19]:
def likelihood(X_train, Y_train, feat_col, feat_val, label):
    X_train = X_train[Y_train == label]
    numerator = np.sum(X_train[:, feat_col] == feat_val)
    denominator = np.sum(Y_train == label)
    return numerator/denominator

In [26]:
def posterior(X_train, Y_train, X_test):
    classes = np.unique(Y_train)
    post_prob = []
    
    for label in classes:
        cond_prob = 1
        
        for i in range(X_train.shape[1]):
            prob = likelihood(X_train, Y_train, i, X_test[i], label)
            cond_prob *= prob
            
        prior = prior_prob(Y_train, label)
        posterior_prob = cond_prob * prior
        post_prob.append(posterior_prob)
        
    return np.argmax(post_prob), post_prob

In [27]:
v = np.array([1,21,2,1,2,1,2,3,1,1,5,3,21,5,6,1,1,5,2])
print(np.argmax(v))

1


In [31]:
print(posterior(train_X, train_Y, test_X[-1]), test_Y[-1])

(0, [1.833011332398652e-27, 0.0]) 0


In [34]:
corr = 0
for i in range(test_X.shape[0]):
    pred, _ = posterior(train_X, train_Y, test_X[i])
    if pred == test_Y[i]:
        corr += 1
    else:
        pass
print(f'Accuracy: {corr/test_X.shape[0]}')

Accuracy: 0.940923076923077
